# Fast selection on object id

Little demo of finding an object by its ID in hipscat, if you don't know the radec.

Start by looking at the hipscat-library index catalog.

In [15]:
from hipscat.catalog.index.index_catalog import IndexCatalog
import pandas as pd

catalog_index = IndexCatalog.read_from_hipscat("/astro/users/mmd11/data/gaia_index")
catalog_index

These are a handful of GAIA designations that are in just three partitions. Let's check the index catalog, and see if we can reduce this list of IDs into just those pixels of interest.

In [16]:
ids = ["Gaia DR3 6350084614282952448", "Gaia DR3 4684296460655809408",
      "Gaia DR3 4684296460655681280", "Gaia DR3 6350084408124522368",
      "Gaia DR3 6379030430412267264", "Gaia DR3 6397962092201358080"]

In [17]:
%%time
partitions = catalog_index.loc_partitions(ids)
partitions

CPU times: user 2.7 s, sys: 578 ms, total: 3.27 s
Wall time: 1.34 s


array([Order: 5, Pixel: 8320, Order: 3, Pixel: 705, Order: 3, Pixel: 708],
      dtype=object)

Now let's try that with lsdb, and fetch the full rows of these objects.

In [18]:
%%time
import lsdb

gaia = lsdb.read_hipscat("/data3/epyc/data3/hipscat/test_catalogs/gaia_symbolic")

CPU times: user 3.24 s, sys: 315 ms, total: 3.56 s
Wall time: 3.55 s


In [19]:
%%time
index_search_df = gaia.index_search(ids, catalog_index).compute()
index_search_df

CPU times: user 9.32 s, sys: 18.7 s, total: 28 s
Wall time: 3.31 s


,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Npix,Dir
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
9368592723742818304,1636148068921376768,Gaia DR3 4684296460655681280,4684296460655681280,935088391,2016.0,7.756788,0.075319,-75.621828,0.069098,0.249487,...,0.0050,0.0010,0.0130,0.0027,0.0005,0.0071,MARCS,5,8320,8320
9368592758102556672,1636148068921376768,Gaia DR3 4684296460655809408,4684296460655809408,930596526,2016.0,7.741220,3.736654,-75.619753,3.646013,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,5,8320,8320
12714633518777368576,1636148068921376768,Gaia DR3 6350084408124522368,6350084408124522368,64296681,2016.0,359.838284,0.249825,-83.921970,0.239047,0.656007,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,705,704
12714633591791812608,1636148068921376768,Gaia DR3 6350084614282952448,6350084614282952448,1750910513,2016.0,359.435728,0.510138,-83.921882,0.492162,-0.272154,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,705,704
12757037060875878400,1636148068921376768,Gaia DR3 6379030430412267264,6379030430412267264,1257926240,2016.0,348.261377,0.101749,-73.622147,0.093109,0.199036,...,0.0050,0.0012,0.0129,0.0027,0.0006,0.0070,MARCS,3,708,708
12771626613808824320,1636148068921376768,Gaia DR3 6397962092201358080,6397962092201358080,150735067,2016.0,336.476393,0.079348,-67.482415,0.092240,0.239489,...,0.0021,0.0004,0.0073,0.0011,0.0002,0.0039,MARCS,3,708,708


In [21]:
%%time
full_scan_df = gaia._ddf["designation"].isin(ids).compute()
full_scan_df